本文主要以kaggle一次二分类竞赛数据为例，深入研究改进forest

LayerDTree是LayerForest的精简版，是对Forest进一步探索的基础
注释比较详细
- 数据地址：https://www.kaggle.com/c/porto-seguro-safe-driver-prediction
- 数据特点：
  - 非常不均衡、缺失值严重、二分类
- 包含内容：
  1. 数据获取
  2. 模型应用
  3. 结果分析

# Stage-1：获取数据

In [1]:
import os
import numpy as np
import pandas as pd
import time

In [2]:
df_train = pd.read_csv('./datasetes/driver/train.csv')
df_test = pd.read_csv('./datasetes/driver/test.csv')

In [3]:
print(df_train[df_train.target==1].shape)
print(df_train[df_train.target==0].shape)
df_train[df_train.target==1].shape[0]/df_train[df_train.target==0].shape[0]

(21694, 59)
(573518, 59)


0.037826188541597645

In [4]:
target_train = df_train['target'].values
id_train = df_train['id'].values
id_test = df_test['id'].values

In [5]:
df_train = df_train.drop(['target','id'],axis=1)
df_test = df_test.drop(['id'], axis = 1)
# combine = pd.concat([df_train,df_test],axis=0)

In [6]:
print ("The train shape is:",df_train.shape)
print ('The test shape is:',df_test.shape)
# print ('The combine shape is:',combine.shape)

The train shape is: (595212, 57)
The test shape is: (892816, 57)


# Stage-2：模型应用

In [42]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedShuffleSplit
import ForestUtils
import time

In [10]:
def gini(actual, pred, cmpcol = 0, sortcol = 1):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=np.float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses
    
    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)

def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

In [70]:
def TrainModelLayer(X, y):
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.15, random_state=9487)
    split_obj = tuple(sss.split(X, y))
    train_index, test_index = split_obj[0]
    X_train, X_valid = X[train_index], X[test_index]
    y_train, y_valid = y[train_index], y[test_index]
    print("X_train.shape, y_train.shape:"+str(X_train.shape)+str(y_train.shape))
    print("X_valid.shape, y_valid.shape:"+str(X_valid.shape)+str(y_valid.shape))
    
    clf = DecisionTreeClassifier(max_depth=10, random_state=1024, min_samples_leaf=100, #max_leaf_nodes=100,
#                                  n_estimators=2, n_jobs=8, oob_score=True, verbose=1, boostrap=False,
                                 criterion='gini')
#     print("train start:"+str(clf))
#     DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, 
#                            min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, 
#                            min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, presort=False)
    clf = clf.fit(X_train, y_train)
#     print("train success")

    p_train = clf.predict_proba(X)
    p_train = [item[1] for item in p_train]
    p_train = np.array(p_train)
    print("data nor-gini:"+str(gini_normalized(y, p_train)))
    
    p_train = clf.predict_proba(X_train)
    p_train = [item[1] for item in p_train]
    p_train = np.array(p_train)
    print("train nor-gini:"+str(gini_normalized(y_train, p_train)))
    
    p_valid = clf.predict_proba(X_valid)
    p_valid = [item[1] for item in p_valid]
    p_valid = np.array(p_valid)
    print("vaild nor-gini:"+str(gini_normalized(y_valid, p_valid)))

    # Predict on our test data
#     p_test = clf.predict_proba(test)
#     p_test = [item[1] for item in p_test]
#     p_test = np.array(p_test)

    
    return clf, X_train, X_valid, y_train, y_valid

In [33]:
def LayerPerformance(clf, X_train):
    impurity = clf.tree_.impurity
    threshold = clf.tree_.threshold
    impurity_index = np.argsort(impurity, axis=0)  
    
    for id in impurity_index:
        if threshold[id] != -2: continue
        ForestUtils.print_node_info(clf, id)
        for item in X_train:
            sample_list = [item]
            leave_id = clf.apply(sample_list)[0]
            if leave_id == id:
                ForestUtils.print_decision_path_ofsample(clf, item)
                break
        print()

In [170]:
def DataSplit(estimator, X_train, y_train):
    data_id_list = np.array(list(range(len(X_train))))
    # S0 对于一个决策树返回所有叶子节点的gini排序计算平均
    impurity = estimator.tree_.impurity
    # impurity_index = np.argsort(impurity, axis=0)  
    mean_imp = np.mean(impurity, axis=0)
    print("mean of all impurity:" + str(mean_imp))

    # S1 通过gini获取相关节点id
    # pass_node_num = impurity[impurity[impurity_index] < mean_imp].shape[0]
    # pass_node_id_lt = impurity_index[:pass_node_num]
    pass_node_id_lt = np.where(impurity < mean_imp)[0]
    print("pass node id shape:" + str(pass_node_id_lt.shape))

    # S2 通过node id获取相关数据
    node_id_lt = estimator.apply(X_train)
    # pass_data_id_lt = []
    # for i, node_id in enumerate(node_id_lt):
    #     if node_id in pass_node_id_lt:
    #         pass_data_id_lt.append(i)
    # len(pass_data_id_lt)
    node_id_mask = np.isin(node_id_lt, pass_node_id_lt)
#     print(node_id_mask, node_id_mask.shape)
#     pass_data_id_lt = data_id_list[node_id_mask]
#     print(pass_data_id_lt, pass_data_id_lt.shape)
#     print("pass data id shape:" + str(pass_data_id_lt.shape))

    # S3 通过pass data id分割数据
#     pass_data_mask = np.isin(data_id_list, pass_data_id_lt)
    pass_data_mask = node_id_mask
    X_train_pass = X_train[pass_data_mask]
    y_train_pass = y_train[pass_data_mask]
    X_train_np = X_train[~pass_data_mask]
    y_train_np = y_train[~pass_data_mask]
#     print("pass data shape:" + str(X_train_pass.shape), y_train_pass[y_train_pass==1].shape)
#     print("not pass data shape:" + str(X_train_np.shape), X_train_np[X_train_np==1].shape)
#     print("all data shape:" + str(X_train_pass.shape[0]+X_train_np.shape[0]))
    
    return X_train_pass, y_train_pass, X_train_np, y_train_np

In [324]:
def TestDataSplit(estimator, X_test):
    # S0 对于一个决策树返回所有叶子节点的gini排序计算平均
    impurity = estimator.tree_.impurity
    mean_imp = np.mean(impurity, axis=0)
    print("mean of all impurity:" + str(mean_imp))

    # S1 通过gini获取相关节点id
    pass_node_id_lt = np.where(impurity < mean_imp)[0]
    print("pass node id shape:" + str(pass_node_id_lt.shape))

    # S2 通过node id获取相关数据
    node_id_lt = estimator.apply(X_test)
    node_id_mask = np.isin(node_id_lt, pass_node_id_lt)
    
    # S3 输出预测结果
    p_test = estimator.predict_proba(X_test)
    p_test = [item[1] for item in p_test]
    p_test = np.array(p_test)
    
    return node_id_mask, p_test 

In [175]:
train = np.array(df_train)
test = np.array(df_test)
X = train
y = target_train

In [131]:
# clf, X_train, X_valid, y_train, y_valid = TrainModelLayer(X, y)
# clf

X_train.shape, y_train.shape:(505930, 57)(505930,)
X_valid.shape, y_valid.shape:(89282, 57)(89282,)
data nor-gini:0.298615876168
train nor-gini:0.316690280626
vaild nor-gini:0.195871634709


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=100, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=1024,
            splitter='best')

In [81]:
# LayerPerformance(clf, X_train)

In [103]:
# X_train_pass, y_train_pass, X_train_np, y_train_np = DataSplit(clf, X, y)

mean of all impurity:0.116536678108
pass node id shape:(466,)
pass data id shape:(537350,)
[ 92  98 114 135 195  94 172 195 648 192] (537350,)
595212
(595212,) (213,)
pass data shape:(213, 57) (9,)
not pass data shape:(594999, 57) (6788379,)
all data shape:595212


In [83]:
# len(train)

In [155]:
# t_y_train = np.array([[item] for item in y_train])
# mask_y_train_1 = np.where(np.any(t_y_train==1, axis=1))[0]
# mask_y_train_0 = np.where(np.any(t_y_train==0, axis=1))[0]

In [284]:
# len(test)

In [325]:
train = np.array(df_train)
test = np.array(df_test)
test_y = np.array(([0.0] * len(test)))
node_id_mask = np.array([False] * len(test))
X = train
y = target_train
X_test = test

# 均衡数据进行layer
X_train_np = X
y_train_np = y
maxlayer = 100
layer = 0

while 1:
    layer += 1
    print()
    print("layer:", layer)
    X = X_train_np
    y = y_train_np
    
    # S1 均衡数据
    positive_mask = np.where(y == 1)[0]
    negative_mask = np.where(y == 0)[0][:len(positive_mask)]
    guest_mask = np.where(y == 0)[0][len(positive_mask):]
    guest_data_x = X[guest_mask]
    guest_data_y = y[guest_mask]
    train_mask = np.hstack((positive_mask, negative_mask))
    train_data_x = X[train_mask]
    train_data_y = y[train_mask]
    
    # S2 训练模型并分割数据
    clf, X_train, X_valid, y_train, y_valid = TrainModelLayer(train_data_x, train_data_y)
    X_train_pass, y_train_pass, X_train_np, y_train_np = DataSplit(clf, train_data_x, train_data_y)
    print("not pass data shape:" + str(X_train_np.shape), y_train_np[y_train_np==1].shape)
    X_train_np = np.vstack((X_train_np, guest_data_x))
    y_train_np = np.hstack((y_train_np, guest_data_y))
    print("pass data shape:" + str(X_train_pass.shape), y_train_pass[y_train_pass==1].shape)
    print("next train data shape:" + str(X_train_np.shape), y_train_np[y_train_np==1].shape)
    print("all data shape:" + str(X_train_pass.shape[0]+X_train_np.shape[0]))
    
    # S3 输出测试数据结果
    X_test_np = X_test[~node_id_mask]
    pass_node_id_mask, p_test = TestDataSplit(clf, X_test_np)
    node_id_mask_index = np.where(node_id_mask == False)[0]
    pass_node_id_mask_index = node_id_mask_index[pass_node_id_mask]
    test_y[pass_node_id_mask_index] = p_test[pass_node_id_mask]
#     print(test_y[~node_id_mask][pass_node_id_mask])
    time.sleep(1)
    
    if X_train_np.shape[0] < 1000 or layer > maxlayer or y_train_np[y_train_np==1].shape[0] <= 10:
#         test_y[~node_id_mask][~pass_node_id_mask] = p_test[~pass_node_id_mask]
        pass_node_id_mask_index = node_id_mask_index[~pass_node_id_mask]
        test_y[pass_node_id_mask_index] = p_test[~pass_node_id_mask]
        break
        
    node_id_mask[~node_id_mask] = pass_node_id_mask


layer: 1
X_train.shape, y_train.shape:(36879, 57)(36879,)
X_valid.shape, y_valid.shape:(6509, 57)(6509,)
data nor-gini:0.311249192135
train nor-gini:0.318810690676
vaild nor-gini:0.197394580887
mean of all impurity:0.462394526573
pass node id shape:(119,)
not pass data shape:(29596, 57) (14703,)
pass data shape:(13792, 57) (6991,)
next train data shape:(581420, 57) (14703,)
all data shape:595212
mean of all impurity:0.462394526573
pass node id shape:(119,)

layer: 2
X_train.shape, y_train.shape:(24995, 57)(24995,)
X_valid.shape, y_valid.shape:(4411, 57)(4411,)
data nor-gini:0.241508564816
train nor-gini:0.248922343379
vaild nor-gini:0.0727091440989
mean of all impurity:0.478016777264
pass node id shape:(87,)
not pass data shape:(18850, 57) (9374,)
pass data shape:(10556, 57) (5329,)
next train data shape:(570864, 57) (9374,)
all data shape:581420
mean of all impurity:0.478016777264
pass node id shape:(87,)

layer: 3
X_train.shape, y_train.shape:(15935, 57)(15935,)
X_valid.shape, y_val

In [330]:
test_y[test_y == 0].shape

(5159,)

In [296]:
sub = pd.DataFrame(test_y, columns=['target'])
sub['id'] = id_test
sub.head()

,target,id
0,0.016134,0
1,0.025643,1
2,0.029184,2
3,0.022739,3
4,0.044334,4


In [297]:
import datetime
file_pre = datetime.datetime.now().strftime('%m_%d_%H_%M')
sub.to_csv("./submission/LayerDTree_"+file_pre+".csv", index=False)

In [326]:
train = np.array(df_train)
test = np.array(df_test)
X = train
y = target_train
X_test = test
test_y = np.array(([0.0] * len(test)))
node_id_mask = np.array([False] * len(test))

# 不均衡数据进行layer
X_train_np = X
y_train_np = y
maxlayer = 100
layer = 0

counter = 0
while 1:
    layer += 1
    print()
    print("layer:", layer)
    X = X_train_np
    y = y_train_np
    clf, X_train, X_valid, y_train, y_valid = TrainModelLayer(X, y)
    X_train_pass, y_train_pass, X_train_np, y_train_np = DataSplit(clf, X, y)
    print("pass data shape:" + str(X_train_pass.shape), y_train_pass[y_train_pass==1].shape)
    print("not pass data shape:" + str(X_train_np.shape), y_train_np[y_train_np==1].shape)
    print("all data shape:" + str(X_train_pass.shape[0]+X_train_np.shape[0]))
#     time.sleep(1)
    
    
    # S3 输出测试数据结果
    X_test_np = X_test[~node_id_mask]
    pass_node_id_mask, p_test = TestDataSplit(clf, X_test_np)
    print(p_test[p_test == 0].shape)
    node_id_mask_index = np.where(node_id_mask == False)[0]
    pass_node_id_mask_index = node_id_mask_index[pass_node_id_mask]
    test_y[pass_node_id_mask_index] = p_test[pass_node_id_mask]
#     print(test_y[~node_id_mask][pass_node_id_mask])
    counter += pass_node_id_mask[pass_node_id_mask == True].shape[0]
    print(p_test[pass_node_id_mask][p_test[pass_node_id_mask] == 0].shape)
    print(counter, len(test), len(test)-counter)
    time.sleep(1)
    
    if X_train_np.shape[0] < 1000 or layer > maxlayer or y_train_np[y_train_np==1].shape[0] <= 10:
#         test_y[~node_id_mask][~pass_node_id_mask] = p_test[~pass_node_id_mask]
        pass_node_id_mask_index = node_id_mask_index[~pass_node_id_mask]
        test_y[pass_node_id_mask_index] = p_test[~pass_node_id_mask]
#         print(test_y[~node_id_mask][~pass_node_id_mask])
        counter += pass_node_id_mask[pass_node_id_mask == False].shape[0]
        print(p_test[~pass_node_id_mask][p_test[~pass_node_id_mask] == 0].shape)
        print(counter, len(test), len(test)-counter)
        break
        
    node_id_mask[~node_id_mask] = pass_node_id_mask
#     break
#     if X_train_np.shape[0] < 1000 or layer > maxlayer:
#         break


layer: 1
X_train.shape, y_train.shape:(505930, 57)(505930,)
X_valid.shape, y_valid.shape:(89282, 57)(89282,)
data nor-gini:0.298615876168
train nor-gini:0.316690280626
vaild nor-gini:0.195871634709
mean of all impurity:0.116536678108
pass node id shape:(466,)
pass data shape:(537350, 57) (16454,)
not pass data shape:(57862, 57) (5240,)
all data shape:595212
mean of all impurity:0.116536678108
pass node id shape:(466,)
(4260,)
(4260,)
806137 892816 86679

layer: 2
X_train.shape, y_train.shape:(49182, 57)(49182,)
X_valid.shape, y_valid.shape:(8680, 57)(8680,)
data nor-gini:0.250215007452
train nor-gini:0.288132309712
vaild nor-gini:0.034379188368
mean of all impurity:0.181957831189
pass node id shape:(157,)
pass data shape:(42088, 57) (2957,)
not pass data shape:(15774, 57) (2283,)
all data shape:57862
mean of all impurity:0.181957831189
pass node id shape:(157,)
(704,)
(704,)
869416 892816 23400

layer: 3
X_train.shape, y_train.shape:(13407, 57)(13407,)
X_valid.shape, y_valid.shape:(23

### Todo list
- 树结构设计（完成）
- 通过gini对数据分割（完成）
- 全局测试集
- 输出结果集
- 打印信息增加pass data的比例
- 防止过拟合
- 对pass data的进一步处理
- 先进行数据均衡化是不是更快一些